In [7]:
from glob import glob
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import math
import os

In [122]:
def read_apple(path):
    # First check if Apple Data exists in the file
    file = open(path, 'r')
    if "Apple Heart Rate" in file.readline():
        apple_data = pd.read_csv(path, usecols=['Time', 'Apple Heart Rate'], parse_dates=['Time'], infer_datetime_format=True).dropna()
        apple_data['Time'] = apple_data['Time'].apply(lambda x: x.timestamp())
        start = apple_data.iloc[0,0]
        apple_data['Time'] = apple_data['Time'].apply(lambda x: int(x - start))
        apple_data = apple_data.values 
    else:
        #print("No Apple Data")
        apple_data = None
        start = None
    file.close()
    return apple_data, start

In [3]:
def read_garmin(path):
    # First check if Garmin Data exists in the file
    file = open(path, 'r')
    if "Garmin Heart Rate" in file.readline():
        garmin_data = pd.read_csv(path, usecols=['Time', 'Garmin Heart Rate'], parse_dates=['Time'], infer_datetime_format=True).dropna()
        garmin_data['Time'] = garmin_data['Time'].apply(lambda x: x.timestamp())
        start = garmin_data.iloc[0,0]
        garmin_data['Time'] = garmin_data['Time'].apply(lambda x: int(x - start))
        garmin_data = garmin_data.values 
    else:
        #print("No Garmin Data")
        garmin_data = None
        start = None
    file.close()
    return garmin_data, start

In [5]:
def read_fitbit(path):
    # First check if Fitbit Data exists in the file
    file = open(path, 'r')
    if "Fitbit Heart Rate" in file.readline():
        fitbit_data = pd.read_csv(path, usecols=['Time', 'Fitbit Heart Rate'], parse_dates=['Time'], infer_datetime_format=True).dropna()
        if fitbit_data.shape[0] > 0:
            fitbit_data['Time'] = fitbit_data['Time'].apply(lambda x: x.timestamp())
            start = fitbit_data.iloc[0,0]
            fitbit_data['Time'] = fitbit_data['Time'].apply(lambda x: int(x - start))
            fitbit_data = fitbit_data.values 
        else:
            fitbit_data=None
            start = None
    else:
        #print("No Fitbit Data")
        fitbit_data = None
        start = None
    file.close()
    return fitbit_data, start

In [3]:
def hr_metrics(some_data, start_time, day_begin, day_end):
    j = 0
    output = {}
    # Iterate through data
    for i in range(some_data.shape[0]):
        # Once 15 seconds have passed can begin the 31 second aggregation
        if some_data[i,0] >= 16 and some_data[i,0] <= some_data[-1,0] - 15:
            # Get the 15 seconds worth of data before pint
            j = i
            while j >= 0 and some_data[j,0] > some_data[i,0] - 15:
                j -= 1
            # Get 15 seconds worth of data after the point
            k = i

            while k < some_data.shape[0] and some_data[k,0] <= some_data[i,0] + 15:
                k += 1
            # Get window of 31 seconds centered at i
            window = some_data[j+1:k, 1]
            # Calculate MEAN
            win_mean = np.mean(window)
            # Calculate STD
            win_std = np.std(window)
            # Calculate Coeficient of Variation
            win_cov = win_std / win_mean
            # Calculate Min
            win_min = np.min(window)
            # Calculate Max
            win_max = np.max(window)
            output[some_data[i,0]] = [win_mean, win_std, win_cov, win_min, win_max]
            #print(f"Window if time is {some_data[i,0]}: \n{window}\n")

    output_df = pd.DataFrame.from_dict(output, orient='index').reset_index()
    output_df.columns=['Time', 'Heart Rate Mean', 'Heart Rate STD', 'Heart Rate Coefficient of Variation', 'Heart Rate Min',
                       'Heart Rate Max']
    output_df['Time'] = output_df['Time'].apply(lambda x: datetime.fromtimestamp(x+start_time))
    if day_begin <= output_df.iloc[0,0] <= day_end:
        output_df['Time'] = output_df['Time'].apply(lambda x: x + timedelta(hours=4))
    else:
        output_df['Time'] = output_df['Time'].apply(lambda x: x + timedelta(hours=5))
    
    return output_df


## Run code below to calcuate Apple HR Metrics

In [124]:
pa_dir = "V:/ACOI/R01 - W4K/3_PA protocol/1_Participants/"
out_dir = "C:/Users/ntindall/Wearables/Physical Activity/Rahul_Accelerometer_Metrics/"

ds_begin = datetime(year=2022, month=3, day=13)
ds_end = datetime(year=2022, month=11, day=6)

participant_paths = glob(pa_dir + "[0-9][0-9][0-9][0-9]/*_agg.csv")
total_participant = len(participant_paths)
curr_participant = 1

for participant in participant_paths:
    participant_id = participant[len(pa_dir):len(pa_dir)+4]
    print(f"Processing Participant {participant_id} {curr_participant}/{total_participant}")
    # Read in data
    apple_hr, start = read_hr(participant)
    if apple_hr is not None:
        # Calculate Metrics and write to file
        hr_df = hr_metrics(apple_hr, start, ds_begin, ds_end)
        # Define Accel path 
        accel_path = out_dir +"Accel_Metrics/" + participant_id + "_accel_metrics.csv"
        # Read in data 
        accel_df = pd.read_csv(accel_path, parse_dates=['Time'], infer_datetime_format=True)
        # Merge to data sets then write to file
        out_path = out_dir + "Combined_Metrics/"+ participant_id + "_combined_metrics.csv"
        accel_df.merge(hr_df, on='Time', how='left').to_csv(out_path, index=False)

        

    else:
        print(f"No Apple Data for {participant_id}")
    
    curr_participant += 1      

Processing Participant 0118 1/162
Processing Participant 0231 2/162
Processing Participant 0444 3/162
Processing Participant 0601 4/162
Processing Participant 0632 5/162
Processing Participant 0695 6/162
Processing Participant 0819 7/162
No Apple Data for 0819
Processing Participant 0967 8/162
Processing Participant 1760 9/162
No Apple Data for 1760
Processing Participant 1772 10/162
No Apple Data for 1772
Processing Participant 1779 11/162
Processing Participant 1799 12/162
No Apple Data for 1799
Processing Participant 1854 13/162
Processing Participant 1974 14/162
Processing Participant 1991 15/162
Processing Participant 2007 16/162
Processing Participant 2031 17/162
No Apple Data for 2031
Processing Participant 2069 18/162
Processing Participant 2113 19/162
Processing Participant 2186 20/162
Processing Participant 2187 21/162
Processing Participant 2271 22/162
Processing Participant 2272 23/162
Processing Participant 2321 24/162
Processing Participant 2355 25/162
Processing Particip

## Run code below to calculate Garmin HR Metrics

In [8]:
pa_dir = "V:/ACOI/R01 - W4K/3_PA protocol/1_Participants/"
out_dir = "C:/Users/ntindall/Wearables/Physical Activity/Rahul_Accelerometer_Metrics/Garmin_Metrics/"

ds_begin = datetime(year=2022, month=3, day=13)
ds_end = datetime(year=2022, month=11, day=6)

participant_paths = glob(pa_dir + "[0-9][0-9][0-9][0-9]/*_agg.csv")
total_participant = len(participant_paths)
curr_participant = 1

for participant in participant_paths:
    participant_id = participant[len(pa_dir):len(pa_dir)+4]
    print(f"Processing Participant {participant_id} {curr_participant}/{total_participant}")
    # Read in data
    garmin_hr, start = read_garmin(participant)
    if garmin_hr is not None:
        # Calculate Metrics and write to file
        hr_df = hr_metrics(garmin_hr, start, ds_begin, ds_end)
        # Define Accel path 
        accel_path = out_dir +"Accel_Metrics/" + participant_id + "_accel_metrics.csv"
        # Read in data 
        accel_df = pd.read_csv(accel_path, parse_dates=['Time'], infer_datetime_format=True)
        # Merge to data sets then write to file
        out_path = out_dir + "Combined_Metrics/"+ participant_id + "_combined_metrics.csv"
        accel_df.merge(hr_df, on='Time', how='left').to_csv(out_path, index=False)

        

    else:
        print(f"No Garmin Data for {participant_id}")
    
    curr_participant += 1   

Processing Participant 0118 1/184


NameError: name 'read_garmin' is not defined

## Run code below to calculate Fitbit HR Metrics

In [10]:
pa_dir = "V:/ACOI/R01 - W4K/3_PA protocol/1_Participants/"
out_dir = "C:/Users/ntindall/Wearables/Physical Activity/Rahul_Accelerometer_Metrics/Fitbit_Metrics/"

ds_begin = datetime(year=2022, month=3, day=13)
ds_end = datetime(year=2022, month=11, day=6)

participant_paths = glob(pa_dir + "[0-9][0-9][0-9][0-9]/*_agg.csv")
total_participant = len(participant_paths)
curr_participant = 1

for participant in participant_paths:
    participant_id = participant[len(pa_dir):len(pa_dir)+4]
    print(f"Processing Participant {participant_id} {curr_participant}/{total_participant}")
    # Read in data
    fitbit_hr, start = read_fitbit(participant)
    if fitbit_hr is not None and os.path.isfile(out_dir +"Accel_Metrics/" + participant_id + "_accel_metrics.csv"):
        # Calculate Metrics and write to file
        hr_df = hr_metrics(fitbit_hr, start, ds_begin, ds_end)
        # Define Accel path 
        accel_path = out_dir +"Accel_Metrics/" + participant_id + "_accel_metrics.csv"
        # Read in data 
        accel_df = pd.read_csv(accel_path, parse_dates=['Time'], infer_datetime_format=True)
        # Merge to data sets then write to file
        out_path = out_dir + "Combined_Metrics/"+ participant_id + "_combined_metrics.csv"
        accel_df.merge(hr_df, on='Time', how='left').to_csv(out_path, index=False)

        

    else:
        print(f"No Fitbit Data for {participant_id}")
    
    curr_participant += 1  

Processing Participant 0118 1/184
No Fitbit Data for 0118
Processing Participant 0231 2/184
Processing Participant 0444 3/184
No Fitbit Data for 0444
Processing Participant 0601 4/184
No Fitbit Data for 0601
Processing Participant 0618 5/184
No Fitbit Data for 0618
Processing Participant 0632 6/184
Processing Participant 0695 7/184
Processing Participant 0819 8/184
No Fitbit Data for 0819
Processing Participant 0967 9/184
No Fitbit Data for 0967
Processing Participant 1333 10/184
No Fitbit Data for 1333
Processing Participant 1760 11/184
No Fitbit Data for 1760
Processing Participant 1772 12/184
Processing Participant 1779 13/184
No Fitbit Data for 1779
Processing Participant 1799 14/184
Processing Participant 1854 15/184
No Fitbit Data for 1854
Processing Participant 1974 16/184
No Fitbit Data for 1974
Processing Participant 1991 17/184
No Fitbit Data for 1991
Processing Participant 2007 18/184
No Fitbit Data for 2007
Processing Participant 2031 19/184
Processing Participant 2069 20/1

Processing Participant 2801 161/184
No Fitbit Data for 2801
Processing Participant 2812 162/184
Processing Participant 2813 163/184
Processing Participant 2814 164/184
Processing Participant 2815 165/184
Processing Participant 2816 166/184
Processing Participant 2817 167/184
Processing Participant 2818 168/184
Processing Participant 2819 169/184
No Fitbit Data for 2819
Processing Participant 2820 170/184
Processing Participant 2821 171/184
Processing Participant 2822 172/184
Processing Participant 2823 173/184
Processing Participant 2824 174/184
Processing Participant 2987 175/184
No Fitbit Data for 2987
Processing Participant 2988 176/184
No Fitbit Data for 2988
Processing Participant 2989 177/184
No Fitbit Data for 2989
Processing Participant 2990 178/184
No Fitbit Data for 2990
Processing Participant 2991 179/184
No Fitbit Data for 2991
Processing Participant 2993 180/184
No Fitbit Data for 2993
Processing Participant 2995 181/184
No Fitbit Data for 2995
Processing Participant 2999 